# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 19 2022 11:05AM,253364,10,SO94472,"Senseonics, Incorporated",Released
1,Dec 19 2022 10:55AM,253363,19,8922302,"Graystone, LLC",Released
2,Dec 19 2022 10:55AM,253363,19,8922304,"Graystone, LLC",Released
3,Dec 19 2022 10:44AM,253362,19,60018266,"GUSA Granules USA, Inc.",Released
4,Dec 19 2022 10:43AM,253361,19,MSP216881,"Methapharm, Inc.",Released
5,Dec 19 2022 10:43AM,253361,19,MSP216884,"Methapharm, Inc.",Released
6,Dec 19 2022 10:19AM,253358,15,8921193,"Brookfield Pharmaceuticals, LLC",Released
7,Dec 19 2022 10:19AM,253358,15,8921194,"Brookfield Pharmaceuticals, LLC",Released
8,Dec 19 2022 10:19AM,253358,15,8921195,"Brookfield Pharmaceuticals, LLC",Released
9,Dec 19 2022 10:19AM,253358,15,8921196,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,253359,Released,23
18,253361,Released,2
19,253362,Released,1
20,253363,Released,2
21,253364,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253359,NaN,NaN,23.0
253361,NaN,NaN,2.0
253362,NaN,NaN,1.0
253363,NaN,NaN,2.0
253364,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1.0,1.0,11.0
253340,0.0,4.0,15.0
253341,0.0,5.0,34.0
253344,0.0,0.0,6.0
253346,0.0,23.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1,1,11
253340,0,4,15
253341,0,5,34
253344,0,0,6
253346,0,23,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,1,11
1,253340,0,4,15
2,253341,0,5,34
3,253344,0,0,6
4,253346,0,23,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,1,11
1,253340,,4,15
2,253341,,5,34
3,253344,,,6
4,253346,,23,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated"
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC"
3,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc."
4,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc."
6,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC"
28,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation
90,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC"
113,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation
153,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation
188,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",,,1
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC",,,2
2,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",,,1
3,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc.",,,2
4,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",,,22
5,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,,,62
6,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",,,23
7,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,,,40
8,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation,,,35
9,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation,,,54


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",1,,
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC",2,,
2,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",1,,
3,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc.",2,,
4,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,,
5,Dec 19 2022 10:18AM,253353,10,ISDIN Corporation,62,,
6,Dec 19 2022 10:16AM,253359,15,"Virtus Pharmaceuticals, LLC",23,,
7,Dec 19 2022 10:13AM,253347,10,ISDIN Corporation,40,,
8,Dec 19 2022 10:12AM,253351,10,ISDIN Corporation,35,,
9,Dec 19 2022 10:12AM,253348,10,ISDIN Corporation,54,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",1,,
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC",2,,
2,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",1,,
3,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc.",2,,
4,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",1,NaN,NaN
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC",2,NaN,NaN
2,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",1,NaN,NaN
3,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc.",2,NaN,NaN
4,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",1,0.0,0.0
1,Dec 19 2022 10:55AM,253363,19,"Graystone, LLC",2,0.0,0.0
2,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",1,0.0,0.0
3,Dec 19 2022 10:43AM,253361,19,"Methapharm, Inc.",2,0.0,0.0
4,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2280144,259,18.0,0.0
15,760063,57,23.0,0.0
19,1013418,16,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2280144,259,18.0,0.0
1,15,760063,57,23.0,0.0
2,19,1013418,16,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,259,18.0,0.0
1,15,57,23.0,0.0
2,19,16,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,259.0
1,15,Released,57.0
2,19,Released,16.0
3,10,Executing,18.0
4,15,Executing,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19
Status,,,
Completed,0.0,0.0,1.0
Executing,18.0,23.0,1.0
Released,259.0,57.0,16.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19
0,Completed,0.0,0.0,1.0
1,Executing,18.0,23.0,1.0
2,Released,259.0,57.0,16.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19
0,Completed,0.0,0.0,1.0
1,Executing,18.0,23.0,1.0
2,Released,259.0,57.0,16.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()